Base에서 실행

# Textcuboid


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## 1) Dataload 20newsgroups  data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
df_newsgroup = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/20news_dataset_clear/20newsgroup_preprocessed.csv', sep=';', usecols=['target', 'text_cleaned'])
df_newsgroup.rename(columns={'text_cleaned' : 'text'}, inplace=True)

In [3]:
le = LabelEncoder()
le.fit(df_newsgroup['target'].unique())

LabelEncoder()

In [4]:
df_newsgroup['target'] = le.transform(df_newsgroup['target'])

In [5]:
X = df_newsgroup['text'].astype(str)
# y = tf.keras.utils.to_categorical(df_newsgroup['target'], num_classes=df_newsgroup['target'].nunique())
y=list(df_newsgroup['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df_newsgroup['target'])

In [6]:
# 방법2) 케라스 내장함수 활용
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(y_train)
one_hot_test_labels = to_categorical(y_test)

## 2) textcuboid 분류

In [7]:
textcuboid=np.load('./1-Channel textcuboid_20ng(bert).npy')

In [8]:
textcuboid_test=np.load('./1-Channel textcuboid_test_20ng(bert).npy')

In [9]:
import keras
from tensorflow.keras import layers

In [18]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, verbose=1),
]

In [19]:
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model


input_layer = Input(shape=(155, 768))
conv1 = Conv1D(512, 1, padding='valid', activation='relu')(input_layer)
pooling = GlobalMaxPooling1D()(conv1)

x = Dense(1024, activation='relu')(pooling)
x = Dropout(0.5)(x)
output_layer = Dense(20, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)

In [20]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 155, 768)]        0         
                                                                 
 conv1d_1 (Conv1D)           (None, 155, 512)          393728    
                                                                 
 global_max_pooling1d_1 (Glo  (None, 512)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 1024)              525312    
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 20)                20500     
                                                           

In [22]:
x_train=textcuboid[1000:]
x_val=textcuboid[:1000]
y_train=one_hot_train_labels[1000:]
y_val=one_hot_train_labels[:1000]

In [23]:
history = model.fit(x_train, y_train,callbacks=callbacks, epochs=20,batch_size=256,validation_data=(x_val, y_val))

Epoch 1/20
55/55 [==============================] - 3s 43ms/step - loss: 4.6965 - accuracy: 0.2608 - val_loss: 1.4773 - val_accuracy: 0.5470 - lr: 0.0010
Epoch 2/20
55/55 [==============================] - 2s 36ms/step - loss: 1.4047 - accuracy: 0.5541 - val_loss: 1.1451 - val_accuracy: 0.6630 - lr: 0.0010
Epoch 3/20
55/55 [==============================] - 2s 35ms/step - loss: 1.0564 - accuracy: 0.6654 - val_loss: 0.9747 - val_accuracy: 0.6840 - lr: 0.0010
Epoch 4/20
55/55 [==============================] - 2s 35ms/step - loss: 0.8316 - accuracy: 0.7390 - val_loss: 0.8855 - val_accuracy: 0.7280 - lr: 0.0010
Epoch 5/20
55/55 [==============================] - 2s 36ms/step - loss: 0.6680 - accuracy: 0.7896 - val_loss: 0.8182 - val_accuracy: 0.7380 - lr: 0.0010
Epoch 6/20
55/55 [==============================] - 2s 35ms/step - loss: 0.5313 - accuracy: 0.8316 - val_loss: 0.8154 - val_accuracy: 0.7450 - lr: 0.0010
Epoch 7/20
55/55 [==============================] - 2s 35ms/step - loss: 0.4

In [24]:
model.evaluate(textcuboid_test,one_hot_test_labels)

118/118 [==============================] - 1s 4ms/step - loss: 0.8159 - accuracy: 0.7995


[0.815866231918335, 0.7995220422744751]

In [25]:
from keras.models import load_model
model = load_model('best_model.h5')
model.evaluate(textcuboid_test,one_hot_test_labels)

118/118 [==============================] - 1s 5ms/step - loss: 0.7625 - accuracy: 0.7825


[0.762452244758606, 0.7825278639793396]

In [18]:
np.average([80.0, 80.0, 80.4])

80.16666666666667